In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
import os,time
import glob,sys

from astropy.io import fits
from astropy.visualization import ZScaleInterval

from RetrieveSource import *
from EstimateBackground import *
from TractorTools import *
from Heasarc import *
from PSF import *


### Get image and catalog

In [2]:
# Check defaults of this function to point to correct file paths. 
uvfilter = 'um2'
def get_file(id,galaxy,segment=1,extension=1,filter=uvfilter,file_location = "/home/bethany/Desktop/Swift_Astrometry"):
    labels = ['RAhr','DEdeg','Umag','e_Umag','Bmag','e_Bmag','Vmag','e_Vmag','Imag','e_Imag','Flag','Jmag','e_Jmag','Hmag','e_Hmag','Ksmag','e_Ksmag']
    fname = glob.glob(f"{file_location}/{galaxy.upper()}/sw000{id}00{segment}{filter}_sk_{id}_{segment}_{extension}.new")
    cname = glob.glob(f"{file_location}/{galaxy.upper()}/sw000{id}00{segment}{filter}_sk_{id}_{segment}_{extension}.full.dat")
    HDR = fits.open(fname[0])
    CAT = pd.read_csv(cname[0],delimiter='\s+',names=labels)
    return HDR[0],CAT,cname

hdr,cat,catname = get_file(id=40415,galaxy='smc')

### Run Retrieve Source

In [3]:
start = time.time()

print('---Getting Catalog---')
source = get_meta().with_hdu(hdu=hdr,
                 usno_catalog='smc_anti_match_USNO_inf.dat',
                 optical_catalog=catname[0],
                 Umag_cutoff=np.nan,
                 Bmag_cutoff=np.nan,
                 fits_origin=0,
                 aperture_size=2.5*2,
                 xdim=[0,np.shape(hdr.data)[1]],  #  <---- Full image is selected here.
                 ydim=[0,np.shape(hdr.data)[0]])


print('---Getting Background---')
bkgd = BkgdEstimator(source,n_pix = [20,20])


print('---Getting PSF---')
pix_scale = np.abs(source.cdelt)*3600.

psf_object = psf_fit(pixel_per_arsecond = 1/pix_scale,
                     uvfilter = uvfilter, width = 23).psf


print('---Running Tractor---')

fwhm = 2.5
sigma =  fwhm/2.355
# Restrict by a factor of 4.
sigma = sigma/4.
        
TractorObject = PhotometryTools(source,
                    psf_filename = psf_object,
                    fits_table_sname = f"temp_tractor.fits", 
                    background = bkgd,
                    fit_positions = sigma,
                    threshold = 0.05)
print(f"Time Taken: {(time.time()-start)/60} minutes")

---Getting Catalog---


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55466.000949 from DATE-OBS.
Set MJD-END to 55466.003194 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55466.000949 from DATE-OBS.
Set MJD-END to 55466.003194 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55466.000949 from DATE-OBS.
Set MJD-END to 55466.003194 from DATE-END'. [astropy.wcs.wcs]


INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]


/home/bethany/Desktop/SUMS/Tractor/RetrieveSource.py:350: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drop_cat['phot'] = phot


---Getting Background---
Masked Sigma Clipped Stats Original Data: 0.005121566820889711 0.004719083197414875 0.0034211515448987484


Masked Sigma Clipped Stats Back Sub Data: 0.0009973413637392198 0.0005785162270206491 0.0033424860410239936
---Getting PSF---
22.898403522933
23
Scaling is 1.0044368366976 ArcSeconds per Pixel
Image needs to be 23 pixels to be 23 ArcSeconds in size.
23
Sum Before Normalization  616.9218729335043
Sum After Normalization  0.9999999999999998
---Running Tractor---
dlnp 23105052.92980059
dlnp 18347.04726340482
dlnp 3186.586712796241
dlnp 1693.5203515627654
dlnp 146.67316366615705
dlnp 626.1249456567457
dlnp 885.659102417645
dlnp 704.822346908506
dlnp 713.3266268956941
dlnp 819.0363192379009
dlnp 803.4598793690093
dlnp 588.3010486507555
dlnp 115.5916529004462
dlnp 111.38736693304963
dlnp 10.24907343776431
dlnp 7.780579455895349
dlnp 5.2638288418529555
dlnp 3.6273686923086643
dlnp 0.5805466998135671
dlnp 3.303041668026708
dlnp 3.0128894489025697
dlnp 4.069327338016592
dlnp 4.953421428450383
dlnp 1.0426901943283156
dlnp 6.336309881298803
dlnp 3.067065233597532
dlnp 8.162659903755412
dlnp 1.517